# TF-IDF

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from pathlib import Path  
import glob
import altair as alt

import numpy as np
import regex
import re
from pymorphy2 import MorphAnalyzer

import nltk
from nltk.corpus import stopwords
import stop_words
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('CRA_train_1200(доп).xlsx')
df = df.head(100) # для быстрого теста работы кода
df

,Id,pr_txt,Категория,Уровень рейтинга
0,1.0,Повышение кредитного рейтинга Акционерного об...,A,A
1,2.0,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB
2,3.0,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A
3,4.0,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,AAA,AAA
4,5.0,29 марта 2023 г. Ведущий рейтинговый аналитик ...,BBB,BBB
...,...,...,...,...
95,96.0,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...,AA,AA+
96,97.0,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...,AA,AA-
97,98.0,АО «Союздорстрой» (ГК АВТОБАН) изменило наимен...,A,A
98,99.0,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...,AA,AA-


In [3]:
df['text'] = df['pr_txt'].astype(str).str.zfill(6)
df

,Id,pr_txt,Категория,Уровень рейтинга,text
0,1.0,Повышение кредитного рейтинга Акционерного об...,A,A,Повышение кредитного рейтинга Акционерного об...
1,2.0,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB,«Эксперт РА» подтвердил кредитный рейтинг комп...
2,3.0,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт..."
3,4.0,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,AAA,AAA,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...
4,5.0,29 марта 2023 г. Ведущий рейтинговый аналитик ...,BBB,BBB,29 марта 2023 г. Ведущий рейтинговый аналитик ...
...,...,...,...,...,...
95,96.0,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...,AA,AA+,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...
96,97.0,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...,AA,AA-,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...
97,98.0,АО «Союздорстрой» (ГК АВТОБАН) изменило наимен...,A,A,АО «Союздорстрой» (ГК АВТОБАН) изменило наимен...
98,99.0,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...,AA,AA-,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...


In [4]:
regexp = RegexpTokenizer('\w+')

df['text_token']=df['text'].apply(regexp.tokenize)
df.head(10)

,Id,pr_txt,Категория,Уровень рейтинга,text,text_token
0,1.0,Повышение кредитного рейтинга Акционерного об...,A,A,Повышение кредитного рейтинга Акционерного об...,"[Повышение, кредитного, рейтинга, Акционерного..."
1,2.0,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB,«Эксперт РА» подтвердил кредитный рейтинг комп...,"[Эксперт, РА, подтвердил, кредитный, рейтинг, ..."
2,3.0,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...","[НКР, повысило, кредитный, рейтинг, ООО, ОТЭКО..."
3,4.0,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,AAA,AAA,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,"[Эксперт, РА, присвоил, кредитный, рейтинг, ПА..."
4,5.0,29 марта 2023 г. Ведущий рейтинговый аналитик ...,BBB,BBB,29 марта 2023 г. Ведущий рейтинговый аналитик ...,"[29, марта, 2023, г, Ведущий, рейтинговый, ана..."
5,6.0,Кредитный рейтинг ПАО «ФосАгро» (далее — Комп...,AAA,AAA,Кредитный рейтинг ПАО «ФосАгро» (далее — Комп...,"[Кредитный, рейтинг, ПАО, ФосАгро, далее, Комп..."
6,7.0,«Эксперт РА» повысил кредитный рейтинг ОАО «МР...,AA,AA+,«Эксперт РА» повысил кредитный рейтинг ОАО «МР...,"[Эксперт, РА, повысил, кредитный, рейтинг, ОАО..."
7,8.0,«Эксперт РА» понизил кредитный рейтинг ПАО «М....,A,A,«Эксперт РА» понизил кредитный рейтинг ПАО «М....,"[Эксперт, РА, понизил, кредитный, рейтинг, ПАО..."
8,9.0,«Эксперт РА» повысил кредитный рейтинг компани...,BB,BB+,«Эксперт РА» повысил кредитный рейтинг компани...,"[Эксперт, РА, повысил, кредитный, рейтинг, ком..."
9,10.0,Кредитный рейтинг ООО «МВМ» (далее — Компани...,A,A,Кредитный рейтинг ООО «МВМ» (далее — Компани...,"[Кредитный, рейтинг, ООО, МВМ, далее, Компания..."


In [5]:
# токенизируем данные
df['text_token'] = df['text_token'].apply(lambda x: ' '.join([item for item in x if len(item)>2]))
df.head(10)

,Id,pr_txt,Категория,Уровень рейтинга,text,text_token
0,1.0,Повышение кредитного рейтинга Акционерного об...,A,A,Повышение кредитного рейтинга Акционерного об...,Повышение кредитного рейтинга Акционерного общ...
1,2.0,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB,«Эксперт РА» подтвердил кредитный рейтинг комп...,Эксперт подтвердил кредитный рейтинг компании ...
2,3.0,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",НКР повысило кредитный рейтинг ООО ОТЭКО Портс...
3,4.0,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,AAA,AAA,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,Эксперт присвоил кредитный рейтинг ПАО ФосАгро...
4,5.0,29 марта 2023 г. Ведущий рейтинговый аналитик ...,BBB,BBB,29 марта 2023 г. Ведущий рейтинговый аналитик ...,марта 2023 Ведущий рейтинговый аналитик Юрова ...
5,6.0,Кредитный рейтинг ПАО «ФосАгро» (далее — Комп...,AAA,AAA,Кредитный рейтинг ПАО «ФосАгро» (далее — Комп...,Кредитный рейтинг ПАО ФосАгро далее Компания Ф...
6,7.0,«Эксперт РА» повысил кредитный рейтинг ОАО «МР...,AA,AA+,«Эксперт РА» повысил кредитный рейтинг ОАО «МР...,Эксперт повысил кредитный рейтинг ОАО МРСК Ура...
7,8.0,«Эксперт РА» понизил кредитный рейтинг ПАО «М....,A,A,«Эксперт РА» понизил кредитный рейтинг ПАО «М....,Эксперт понизил кредитный рейтинг ПАО Видео ур...
8,9.0,«Эксперт РА» повысил кредитный рейтинг компани...,BB,BB+,«Эксперт РА» повысил кредитный рейтинг компани...,Эксперт повысил кредитный рейтинг компании ООО...
9,10.0,Кредитный рейтинг ООО «МВМ» (далее — Компани...,A,A,Кредитный рейтинг ООО «МВМ» (далее — Компани...,Кредитный рейтинг ООО МВМ далее Компания МВМ о...


In [6]:
# создадим функцию для лемматизации токенов
stopwords_ru = stopwords.words('russian')
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None


In [7]:
# применим лемматизатор
df['text_lemmatized'] = df['text_token'].apply(lemmatize)
df.head(10)

,Id,pr_txt,Категория,Уровень рейтинга,text,text_token,text_lemmatized
0,1.0,Повышение кредитного рейтинга Акционерного об...,A,A,Повышение кредитного рейтинга Акционерного об...,Повышение кредитного рейтинга Акционерного общ...,"[повышение, кредитный, рейтинг, акционерный, о..."
1,2.0,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB,«Эксперт РА» подтвердил кредитный рейтинг комп...,Эксперт подтвердил кредитный рейтинг компании ...,"[эксперт, подтвердить, кредитный, рейтинг, ком..."
2,3.0,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",НКР повысило кредитный рейтинг ООО ОТЭКО Портс...,"[нкр, повысить, кредитный, рейтинг, ооо, отэко..."
3,4.0,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,AAA,AAA,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,Эксперт присвоил кредитный рейтинг ПАО ФосАгро...,"[эксперт, присвоить, кредитный, рейтинг, пао, ..."
4,5.0,29 марта 2023 г. Ведущий рейтинговый аналитик ...,BBB,BBB,29 марта 2023 г. Ведущий рейтинговый аналитик ...,марта 2023 Ведущий рейтинговый аналитик Юрова ...,"[март, ведущий, рейтинговый, аналитик, юров, а..."
5,6.0,Кредитный рейтинг ПАО «ФосАгро» (далее — Комп...,AAA,AAA,Кредитный рейтинг ПАО «ФосАгро» (далее — Комп...,Кредитный рейтинг ПАО ФосАгро далее Компания Ф...,"[кредитный, рейтинг, пао, фосагро, далее, комп..."
6,7.0,«Эксперт РА» повысил кредитный рейтинг ОАО «МР...,AA,AA+,«Эксперт РА» повысил кредитный рейтинг ОАО «МР...,Эксперт повысил кредитный рейтинг ОАО МРСК Ура...,"[эксперт, повысить, кредитный, рейтинг, оао, м..."
7,8.0,«Эксперт РА» понизил кредитный рейтинг ПАО «М....,A,A,«Эксперт РА» понизил кредитный рейтинг ПАО «М....,Эксперт понизил кредитный рейтинг ПАО Видео ур...,"[эксперт, понизить, кредитный, рейтинг, пао, в..."
8,9.0,«Эксперт РА» повысил кредитный рейтинг компани...,BB,BB+,«Эксперт РА» повысил кредитный рейтинг компани...,Эксперт повысил кредитный рейтинг компании ООО...,"[эксперт, повысить, кредитный, рейтинг, компан..."
9,10.0,Кредитный рейтинг ООО «МВМ» (далее — Компани...,A,A,Кредитный рейтинг ООО «МВМ» (далее — Компани...,Кредитный рейтинг ООО МВМ далее Компания МВМ о...,"[кредитный, рейтинг, ооо, мвм, далее, компания..."


In [8]:
def list_to_str(word_list):
    return ' '.join(word_list)

df['text_lemmatized'] = df['text_lemmatized'].apply(lambda x: list_to_str(x))
df.head(10)

,Id,pr_txt,Категория,Уровень рейтинга,text,text_token,text_lemmatized
0,1.0,Повышение кредитного рейтинга Акционерного об...,A,A,Повышение кредитного рейтинга Акционерного об...,Повышение кредитного рейтинга Акционерного общ...,повышение кредитный рейтинг акционерный общест...
1,2.0,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB,«Эксперт РА» подтвердил кредитный рейтинг комп...,Эксперт подтвердил кредитный рейтинг компании ...,эксперт подтвердить кредитный рейтинг компания...
2,3.0,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",НКР повысило кредитный рейтинг ООО ОТЭКО Портс...,нкр повысить кредитный рейтинг ооо отэко портс...
3,4.0,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,AAA,AAA,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,Эксперт присвоил кредитный рейтинг ПАО ФосАгро...,эксперт присвоить кредитный рейтинг пао фосагр...
4,5.0,29 марта 2023 г. Ведущий рейтинговый аналитик ...,BBB,BBB,29 марта 2023 г. Ведущий рейтинговый аналитик ...,марта 2023 Ведущий рейтинговый аналитик Юрова ...,март ведущий рейтинговый аналитик юров алла мл...
5,6.0,Кредитный рейтинг ПАО «ФосАгро» (далее — Комп...,AAA,AAA,Кредитный рейтинг ПАО «ФосАгро» (далее — Комп...,Кредитный рейтинг ПАО ФосАгро далее Компания Ф...,кредитный рейтинг пао фосагро далее компания ф...
6,7.0,«Эксперт РА» повысил кредитный рейтинг ОАО «МР...,AA,AA+,«Эксперт РА» повысил кредитный рейтинг ОАО «МР...,Эксперт повысил кредитный рейтинг ОАО МРСК Ура...,эксперт повысить кредитный рейтинг оао мрск ур...
7,8.0,«Эксперт РА» понизил кредитный рейтинг ПАО «М....,A,A,«Эксперт РА» понизил кредитный рейтинг ПАО «М....,Эксперт понизил кредитный рейтинг ПАО Видео ур...,эксперт понизить кредитный рейтинг пао видео у...
8,9.0,«Эксперт РА» повысил кредитный рейтинг компани...,BB,BB+,«Эксперт РА» повысил кредитный рейтинг компани...,Эксперт повысил кредитный рейтинг компании ООО...,эксперт повысить кредитный рейтинг компания оо...
9,10.0,Кредитный рейтинг ООО «МВМ» (далее — Компани...,A,A,Кредитный рейтинг ООО «МВМ» (далее — Компани...,Кредитный рейтинг ООО МВМ далее Компания МВМ о...,кредитный рейтинг ооо мвм далее компания мвм о...


In [9]:
tfidf_vectorizer = TfidfVectorizer(min_df = 2)
tfidfmatrix = tfidf_vectorizer.fit_transform(df['text_lemmatized'])
tfidfmatrix_df = pd.DataFrame(tfidfmatrix.toarray(), columns = tfidf_vectorizer.get_feature_names_out(), index = df.index)
tfidfmatrix_df

,абонент,абонентский,абсолютный,август,авиадвигатель,авиастроительный,авиационный,авиация,автоматизация,автомобиль,...,юань,южный,юридический,юров,явиться,явление,являться,ямало,январь,яндекс
0,0.0,0.0,0.040486,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.028486,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.095506,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.02968,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.031699,0.000000,0.000000,0.0,0.0,0.070184,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.074313,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.037924,0.0,0.0,0.056806,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.080421,0.0,0.0,0.0
96,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.083566,0.0,0.0,0.0
97,0.0,0.0,0.049211,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.027096,0.000000,0.0,0.0,0.043281,0.0,0.0,0.0
98,0.0,0.0,0.017980,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.050603,0.0,0.0,0.0


In [10]:
tfidfmatrix_df_full = pd.concat([df, tfidfmatrix_df], axis = 1)
tfidfmatrix_df_full

,Id,pr_txt,Категория,Уровень рейтинга,text,text_token,text_lemmatized,абонент,абонентский,абсолютный,...,юань,южный,юридический,юров,явиться,явление,являться,ямало,январь,яндекс
0,1.0,Повышение кредитного рейтинга Акционерного об...,A,A,Повышение кредитного рейтинга Акционерного об...,Повышение кредитного рейтинга Акционерного общ...,повышение кредитный рейтинг акционерный общест...,0.0,0.0,0.040486,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.028486,0.0,0.0,0.0
1,2.0,«Эксперт РА» подтвердил кредитный рейтинг комп...,BB,BB,«Эксперт РА» подтвердил кредитный рейтинг комп...,Эксперт подтвердил кредитный рейтинг компании ...,эксперт подтвердить кредитный рейтинг компания...,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.095506,0.0,0.0,0.0
2,3.0,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",A,A,"НКР повысило кредитный рейтинг ООО ""ОТЭКО-Порт...",НКР повысило кредитный рейтинг ООО ОТЭКО Портс...,нкр повысить кредитный рейтинг ооо отэко портс...,0.0,0.0,0.000000,...,0.0,0.031699,0.000000,0.000000,0.0,0.0,0.070184,0.0,0.0,0.0
3,4.0,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,AAA,AAA,«Эксперт РА» присвоил кредитный рейтинг ПАО «Ф...,Эксперт присвоил кредитный рейтинг ПАО ФосАгро...,эксперт присвоить кредитный рейтинг пао фосагр...,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.074313,0.0,0.0,0.0
4,5.0,29 марта 2023 г. Ведущий рейтинговый аналитик ...,BBB,BBB,29 марта 2023 г. Ведущий рейтинговый аналитик ...,марта 2023 Ведущий рейтинговый аналитик Юрова ...,март ведущий рейтинговый аналитик юров алла мл...,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.037924,0.0,0.0,0.056806,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96.0,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...,AA,AA+,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...,Эксперт подтвердил кредитный рейтинг ПАО Россе...,эксперт подтвердить кредитный рейтинг пао росс...,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.080421,0.0,0.0,0.0
96,97.0,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...,AA,AA-,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...,Эксперт подтвердил кредитный рейтинг ПАО Транс...,эксперт подтвердить кредитный рейтинг пао тран...,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.083566,0.0,0.0,0.0
97,98.0,АО «Союздорстрой» (ГК АВТОБАН) изменило наимен...,A,A,АО «Союздорстрой» (ГК АВТОБАН) изменило наимен...,Союздорстрой АВТОБАН изменило наименование Инв...,союздорстрой автобан изменить наименование инв...,0.0,0.0,0.049211,...,0.0,0.000000,0.027096,0.000000,0.0,0.0,0.043281,0.0,0.0,0.0
98,99.0,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...,AA,AA-,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...,Эксперт подтвердил кредитный рейтинг ПАО Акрон...,эксперт подтвердить кредитный рейтинг пао акро...,0.0,0.0,0.017980,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.050603,0.0,0.0,0.0


In [11]:
tfidfmatrix_df_full.index = tfidfmatrix_df_full['Уровень рейтинга']
tfidfmatrix_df_full = tfidfmatrix_df_full.drop(columns=["Id","pr_txt", "text",  "Категория","text_token", "text_lemmatized", "Уровень рейтинга"])
tfidfmatrix_df_full

,абонент,абонентский,абсолютный,август,авиадвигатель,авиастроительный,авиационный,авиация,автоматизация,автомобиль,...,юань,южный,юридический,юров,явиться,явление,являться,ямало,январь,яндекс
Уровень рейтинга,,,,,,,,,,,,,,,,,,,,,
A,0.0,0.0,0.040486,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.028486,0.0,0.0,0.0
BB,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.095506,0.0,0.0,0.0
A,0.0,0.0,0.000000,0.02968,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.031699,0.000000,0.000000,0.0,0.0,0.070184,0.0,0.0,0.0
AAA,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.074313,0.0,0.0,0.0
BBB,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.037924,0.0,0.0,0.056806,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AA+,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.080421,0.0,0.0,0.0
AA-,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.083566,0.0,0.0,0.0
A,0.0,0.0,0.049211,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.027096,0.000000,0.0,0.0,0.043281,0.0,0.0,0.0


In [12]:
df_key_words = tfidfmatrix_df_full

In [13]:
key_words = pd.DataFrame()
key_words['Cat'] = df_key_words.index.unique()
key_words['words'] = ''
for i in df_key_words.index.unique():
    key_words.loc[key_words['Cat'] == i, 'words'] = str(df_key_words[df_key_words.index == i].sum().sort_values(ascending = False).head(20).index)[7:-24]
        
key_words

,Cat,words
0,A,"'рейтинг', 'компания', 'кредитный', 'эксперт',..."
1,BB,"'нкр', 'рейтинг', 'компания', 'кредитный', 'оо..."
2,AAA,"'рейтинг', 'эксперт', 'компания', 'кредитный',..."
3,BBB,"'компания', 'рейтинг', 'год', 'кредитный', 'эк..."
4,AA+,"'компания', 'рейтинг', 'год', 'россеть', 'кред..."
5,BB+,"'компания', 'рейтинг', 'эксперт', 'год', 'проф..."
6,BB-,"'компания', 'рейтинг', 'ооо', 'кредитный', 'го..."
7,A-,"'компания', 'рейтинг', 'эксперт', 'кредитный',..."
8,A+,"'рейтинг', 'компания', 'эксперт', 'год', 'кред..."
9,B,"'нра', 'компания', 'рейтинг', 'ооо', 'топливо'..."


In [14]:
key_words.to_excel('key_words.xlsx')

### Визуализация

In [15]:
df_key_words.stack().reset_index()

,Уровень рейтинга,level_1,0
0,A,абонент,0.000000
1,A,абонентский,0.000000
2,A,абсолютный,0.040486
3,A,август,0.000000
4,A,авиадвигатель,0.000000
...,...,...,...
249995,BBB,явление,0.000000
249996,BBB,являться,0.042217
249997,BBB,ямало,0.000000
249998,BBB,январь,0.000000


In [16]:
df_key_words = df_key_words.stack().reset_index()
df_key_words

,Уровень рейтинга,level_1,0
0,A,абонент,0.000000
1,A,абонентский,0.000000
2,A,абсолютный,0.040486
3,A,август,0.000000
4,A,авиадвигатель,0.000000
...,...,...,...
249995,BBB,явление,0.000000
249996,BBB,являться,0.042217
249997,BBB,ямало,0.000000
249998,BBB,январь,0.000000


In [17]:
df_key_words = df_key_words.rename(columns={0:'tfidf', 'Уровень рейтинга': 'rating','level_1': 'term'})
df_key_words

,rating,term,tfidf
0,A,абонент,0.000000
1,A,абонентский,0.000000
2,A,абсолютный,0.040486
3,A,август,0.000000
4,A,авиадвигатель,0.000000
...,...,...,...
249995,BBB,явление,0.000000
249996,BBB,являться,0.042217
249997,BBB,ямало,0.000000
249998,BBB,январь,0.000000


In [18]:
df_key_words.sort_values(by=['rating','tfidf'], ascending=[True,False]).groupby(['rating']).head(10)

,rating,term,tfidf
116863,A,самолёт,0.594554
24969,A,эльдорадо,0.552526
2276,A,уральский,0.546792
22666,A,видео,0.527302
6075,A,нкр,0.519338
...,...,...,...
183213,BBB-,компания,0.212032
182843,BBB-,группа,0.179285
111107,BBB-,облигация,0.176414
110785,BBB-,кредитный,0.173318


In [19]:
top_df_key_words = df_key_words.sort_values(by=['rating','tfidf'], ascending=[True,False]).groupby(['rating']).head(15)
top_df_key_words

,rating,term,tfidf
116863,A,самолёт,0.594554
24969,A,эльдорадо,0.552526
2276,A,уральский,0.546792
22666,A,видео,0.527302
6075,A,нкр,0.519338
...,...,...,...
182530,BBB-,акр,0.141254
110030,BBB-,акр,0.130158
184302,BBB-,рейтинг,0.123115
183831,BBB-,перенос,0.116887


In [20]:
# добавим немного рандомизированности для будущей демонстрации
top_tfidf_plusRand = top_df_key_words.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_df_key_words.shape[0])*0.0001

# задаем будущую визуализацию
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'rating:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["rating"],
)

# настройка heatmap
heatmap = base.mark_rect().encode(
    alt.Color('tfidf:Q', scale=alt.Scale(scheme="viridis"))
)


# добавим цвета для лучшей видимости
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('black'), alt.value('white'))
)

# выводим изображение
(heatmap + text).properties(width = 1500)

alt.LayerChart(...)